In [ ]:
import os
import tkinter as tk
from tkinter import scrolledtext
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, LSTM, Dropout, Dense
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

MODEL_DIR = os.path.join(".", "saved_model")
MODEL_FILE_PATH = os.path.join(MODEL_DIR, "model_traffic_test.h5")

# Definirea coloanelor
columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent',
           'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
           'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
           'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
           'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
           'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
           'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack', 'level']

# Funcție de încărcare și preprocesare a datelor NSL-KDD
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path, delimiter=',', header=None)
    df.columns = columns  # Setează numele coloanelor
    
    X = df.iloc[:, :-1]  # Toate coloanele, mai puțin ultima
    y_raw = df['attack']  # Ultima coloană este eticheta
    
    # Convertim coloanele categorice
    label_encoders = {}
    for column in X.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])
        label_encoders[column] = le
    
    # Convertim etichetele de ieșire
    label_encoder_y = LabelEncoder()
    y = label_encoder_y.fit_transform(y_raw)
    
    # Scala datele
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)
    
    return X_scaled, y, label_encoder_y

# Crearea modelului CNN-LSTM
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Convolution1D(128, 3, padding="same", activation="relu", input_shape=input_shape),
        Convolution1D(128, 3, padding="same", activation="relu"),
        Convolution1D(128, 3, padding="same", activation="relu"),
        MaxPooling1D(pool_size=2),
        LSTM(30),
        Dropout(0.1),
        Dense(num_classes, activation="softmax")
    ])
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model

# Încărcare și pregătire date
X, y, label_encoder_y = load_and_preprocess_data("../Data/nsl-kdd/KDDTrain+.txt")
num_classes = len(np.unique(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificăm dacă modelul este deja salvat
if os.path.exists(MODEL_FILE_PATH):
    model = load_model(MODEL_FILE_PATH, compile=False)
    print("Modelul a fost încărcat din fișierul salvat.")
else:
    os.makedirs(MODEL_DIR, exist_ok=True)
    model = create_cnn_lstm_model(X_train.shape[1:], num_classes)
    model.fit(X_train, y_train, epochs=1, batch_size=64, validation_data=(X_test, y_test))
    model.save(MODEL_FILE_PATH)
    print("Modelul a fost antrenat și salvat în format .h5.")

# Funcție de predicție detaliată
def preprocess_and_predict(data, threshold=50):
    prediction_probs = model.predict(data)[0]
    predicted_index = np.argmax(prediction_probs)
    confidence = prediction_probs[predicted_index] * 100
    
    # Clasificăm ca trafic legitim dacă încrederea este sub prag
    if confidence < threshold:
        return "Trafic Legitim", confidence
    
    # Obțineți categoria specifică de atac și denumirea etichetei
    predicted_category = label_encoder_y.inverse_transform([predicted_index])[0]
    return f"Atac: {predicted_category}", confidence

# Interfața Tkinter pentru analiza traficului
class LiveTrafficAnalyzer:
    def __init__(self, root):
        self.root = root
        self.root.title("Live Internet Traffic Analyzer")
        
        self.label = tk.Label(root, text="Internet Traffic Analysis", font=("Arial", 16))
        self.label.pack(pady=10)

        # Butoane de start și stop
        self.start_button = tk.Button(root, text="Start Analysis", command=self.start_analysis)
        self.start_button.pack(pady=5)
        self.stop_button = tk.Button(root, text="Stop Analysis", command=self.stop_analysis, state="disabled")
        self.stop_button.pack(pady=5)

        # ScrolledText pentru afișarea istoricului
        self.history_text = scrolledtext.ScrolledText(root, width=80, height=15, font=("Arial", 10))
        self.history_text.pack(pady=10)
        self.history_text.insert(tk.END, "Timestamp\t\tPredicted Attack Type\t\tConfidence (%)\n")
        self.history_text.insert(tk.END, "-"*60 + "\n")

        # Flag pentru controlul analizei continue
        self.running = False

    def start_analysis(self):
        self.running = True
        self.start_button.config(state="disabled")
        self.stop_button.config(state="normal")
        self.run_analysis()

    def stop_analysis(self):
        self.running = False
        self.start_button.config(state="normal")
        self.stop_button.config(state="disabled")

    def run_analysis(self):
        if self.running:
            sample_data = X_test[np.random.randint(0, len(X_test))].reshape(1, X_test.shape[1], 1)
            attack_type, confidence = preprocess_and_predict(sample_data)
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            self.history_text.insert(tk.END, f"{current_time}\t{attack_type}\t\t{confidence:.2f}%\n")
            self.history_text.see(tk.END)
            self.root.after(3000, self.run_analysis)

# Inițializarea și rularea aplicației Tkinter
root = tk.Tk()
app = LiveTrafficAnalyzer(root)
root.mainloop()


c:\Users\Here\anaconda3\envs\gputf3.10\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1575/1575 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.8472 - loss: 0.6175 - val_accuracy: 0.9731 - val_loss: 0.1248


Modelul a fost antrenat și salvat în format .h5.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
